# Numerical Schemes

**Goal:** Visualize and compare different advection, diffusion, and time stepping schemes available in NEMO using the 1D advection-diffusion equation with periodic boundary conditions. In the context of NEMO, this refers to tracer variables evolved without external forcing or momentum coupling.

---

## Advection-Diffusion Equation

The general advection-diffusion equation for a tracer concentration $c(\mathbf{x}, t)$ is:

$$
\frac{\partial c}{\partial t} = 
\underbrace{
  -\nabla \cdot (\mathbf{v} c)
}_{\text{advection}}
+
\underbrace{
  \nabla \cdot ( D \nabla c )
}_{\text{diffusion}}
$$

where:
- $c(\mathbf{x}, t)$ is the tracer concentration,
- $\mathbf{v}$ is the velocity field,
- $D$ is the diffusion coefficient.


## Imports

In [1]:
from dash import Dash, dcc, html, Input, Output, State, callback_context
import numpy as np
import plotly.graph_objs as go
from typing import Any, List, Tuple, Dict

## Problem

In [2]:
L: float = 2 * np.pi  # Domain length
u: float = 1.0        # Advection velocity

def initial_condition(x: np.ndarray) -> np.ndarray:
    """
    Returns the initial condition for the tracer concentration as a sine wave.

    Parameters
    ----------
    x : np.ndarray
        Array of spatial coordinates.

    Returns
    -------
    np.ndarray
        Initial tracer concentration at each position in x.
    """
    return np.sin(x)

## Schemes

### Advection

In [3]:
def compute_flux_up1(T: np.ndarray, u: float, dx: float) -> np.ndarray:
    """
    Compute the numerical flux at cell interfaces using the first-order upwind (UP1) advection scheme.

    Parameters
    ----------
    T : np.ndarray
        Array of tracer values at cell centers.
    u : float
        Advection velocity (assumed constant for all cells).
    dx : float
        Grid spacing (not used in this function, but may be required for interface compatibility).

    Returns
    -------
    np.ndarray
        Array of flux values at cell interfaces (length: len(T) + 1).

    Notes
    -----
    - Periodic boundary conditions are applied using np.pad with 'wrap' mode.
    - The upwind direction is chosen based on the sign of u.
    - This function does not modify the input array T.
    """
    T_padded = np.pad(T, (1, 1), mode='wrap')  # Apply periodic boundaries
    flux = np.zeros(len(T) + 1)  # Initialize flux array at interfaces
    for i in range(len(flux)):
        ip = i + 1
        # Select upwind value based on flow direction
        T_half = T_padded[ip - 1] if u > 0 else T_padded[ip]
        flux[i] = u * T_half
    return flux

In [4]:
def compute_flux_up3(T: np.ndarray, u: float, dx: float) -> np.ndarray:
    """
    Compute the numerical flux at cell interfaces using the third-order upwind (UP3) advection scheme.

    Parameters
    ----------
    T : np.ndarray
        Array of tracer values at cell centers.
    u : float
        Advection velocity (assumed constant for all cells).
    dx : float
        Grid spacing (not used in this function, but may be required for interface compatibility).

    Returns
    -------
    np.ndarray
        Array of flux values at cell interfaces (length: len(T) + 1).

    Notes
    -----
    - Periodic boundary conditions are applied using np.pad with 'wrap' mode.
    - The upwind direction is chosen based on the sign of u.
    - This function uses a third-order upwind stencil for higher accuracy.
    - This function does not modify the input array T.
    """
    T_padded = np.pad(T, (2, 2), mode='wrap')  # Apply periodic boundaries
    flux = np.zeros(len(T) + 1)  # Initialize flux array at interfaces
    for i in range(len(flux)):
        ip = i + 2
        # Select upwind-biased value using a third-order stencil based on flow direction
        if u > 0:
            T_half = (2 * T_padded[ip - 1] + 5 * T_padded[ip] - T_padded[ip + 1]) / 6.
        else:
            T_half = (2 * T_padded[ip + 2] + 5 * T_padded[ip + 1] - T_padded[ip]) / 6.
        flux[i] = u * T_half
    return flux

In [5]:
def compute_flux_cen2(T: np.ndarray, u: float, dx: float) -> np.ndarray:
    """
    Compute the numerical flux at cell interfaces using the second-order centered (CEN2) advection scheme.

    Parameters
    ----------
    T : np.ndarray
        Array of tracer values at cell centers.
    u : float
        Advection velocity (assumed constant for all cells).
    dx : float
        Grid spacing (not used in this function, but may be required for interface compatibility).

    Returns
    -------
    np.ndarray
        Array of flux values at cell interfaces (length: len(T) + 1).

    Notes
    -----
    - Periodic boundary conditions are applied using np.pad with 'wrap' mode.
    - The interface value is computed as the average of the two adjacent cell centers (second-order accurate).
    - This function does not modify the input array T.
    """
    T_padded = np.pad(T, (1, 1), mode='wrap')  # Apply periodic boundaries
    flux = np.zeros(len(T) + 1)  # Initialize flux array at interfaces
    for i in range(len(flux)):
        ip = i + 1
        # Use centered average for interface value (second-order accuracy)
        T_half = 0.5 * (T_padded[ip] + T_padded[ip - 1])
        flux[i] = u * T_half
    return flux

In [6]:
def compute_flux_cen4(T: np.ndarray, u: float, dx: float) -> np.ndarray:
    """
    Compute the numerical flux at cell interfaces using the fourth-order centered (CEN4) advection scheme.

    Parameters
    ----------
    T : np.ndarray
        Array of tracer values at cell centers.
    u : float
        Advection velocity (assumed constant for all cells).
    dx : float
        Grid spacing (not used in this function, but may be required for interface compatibility).

    Returns
    -------
    np.ndarray
        Array of flux values at cell interfaces (length: len(T) + 1).

    Notes
    -----
    - Periodic boundary conditions are applied using np.pad with 'wrap' mode.
    - The interface value is estimated using a fourth-order centered finite-difference stencil for higher accuracy.
    - This function does not modify the input array T.
    """
    T_padded = np.pad(T, (2, 2), mode='wrap')  # Apply periodic boundaries
    flux = np.zeros(len(T) + 1)  # Initialize flux array at interfaces
    for i in range(len(flux)):
        ip = i + 2
        # Compute differences for the fourth-order stencil
        delta_i = T_padded[ip] - T_padded[ip - 1]
        delta_ip1 = T_padded[ip + 1] - T_padded[ip]
        # Fourth-order centered interface value
        T_half = T_padded[ip] + (delta_i - delta_ip1) / 6.0
        flux[i] = u * T_half
    return flux

In [7]:
def compute_flux_fct(
    T: np.ndarray,
    u: float,
    dx: float,
    base: str = 'cen4',
    theta: float = 0.5
) -> np.ndarray:
    """
    Compute the numerical flux at cell interfaces using a Flux-Corrected Transport (FCT) scheme.

    The FCT scheme blends a low-order (first-order upwind) and a high-order (centered) flux.

    Parameters
    ----------
    T : np.ndarray
        Array of tracer values at cell centers.
    u : float
        Advection velocity (assumed constant for all cells).
    dx : float
        Grid spacing (not used in this function, but may be required for interface compatibility).
    base : str, optional
        High-order scheme to use ('cen2' or 'cen4'). Default is 'cen4'.
    theta : float, optional
        Blending factor between high-order and low-order flux (0 <= theta <= 1).
        theta=1: fully high-order, theta=0: fully low-order. Default is 0.5.

    Returns
    -------
    np.ndarray
        Array of flux values at cell interfaces (length: len(T) + 1).

    Notes
    -----
    - Periodic boundary conditions are applied using np.pad with 'wrap' mode for the low-order flux.
    - The low-order flux is first-order upwind (UP1).
    - The high-order flux can be second- or fourth-order centered.
    - The final flux is a convex combination of high- and low-order fluxes.
    """
    # Low-order (upwind) flux
    T_padded = np.pad(T, (1, 1), mode='wrap')
    flux_lo = np.zeros(len(T) + 1)
    for i in range(len(flux_lo)):
        ip = i + 1
        T_half = T_padded[ip - 1] if u > 0 else T_padded[ip]
        flux_lo[i] = u * T_half

    # High-order (centered) flux
    if base == 'cen4':
        flux_ho = compute_flux_cen4(T, u, dx)
    elif base == 'cen2':
        flux_ho = compute_flux_cen2(T, u, dx)
    else:
        raise ValueError("FCT base must be 'cen2' or 'cen4'.")

    # Blend the fluxes
    return theta * flux_ho + (1 - theta) * flux_lo

In [8]:
def compute_flux_none(T: np.ndarray, u: float, dx: float) -> np.ndarray:
    """
    Return zero flux at all cell interfaces (i.e., no advection).

    Parameters
    ----------
    T : np.ndarray
        Array of tracer values at cell centers.
    u : float
        Advection velocity (ignored).
    dx : float
        Grid spacing (ignored).

    Returns
    -------
    np.ndarray
        Array of zero flux values at cell interfaces (length: len(T) + 1).

    Notes
    -----
    - This function can be used to disable advection in the numerical scheme.
    - All parameters except T are ignored.
    """
    return np.zeros(len(T) + 1)

### Diffsuion

In [9]:
def laplacian_2nd(T: np.ndarray, dx: float) -> np.ndarray:
    """
    Compute the second-order central difference Laplacian (diffusion operator) for a 1D array.

    Parameters
    ----------
    T : np.ndarray
        Array of values at cell centers.
    dx : float
        Grid spacing.

    Returns
    -------
    np.ndarray
        The second-order approximation of the Laplacian of T.

    Notes
    -----
    - Assumes periodic boundary conditions using np.roll.
    - Formula: (T_{i+1} - 2*T_i + T_{i-1}) / dx^2
    """
    return (np.roll(T, -1) - 2 * T + np.roll(T, 1)) / dx**2

In [10]:
def laplacian_4th(T: np.ndarray, dx: float) -> np.ndarray:
    """
    Compute the fourth-order central difference Laplacian (diffusion operator) for a 1D array.

    Parameters
    ----------
    T : np.ndarray
        Array of values at cell centers.
    dx : float
        Grid spacing.

    Returns
    -------
    np.ndarray
        The fourth-order approximation of the Laplacian of T.

    Notes
    -----
    - Assumes periodic boundary conditions using np.roll.
    - Formula: (-T_{i-2} + 16*T_{i-1} - 30*T_i + 16*T_{i+1} - T_{i+2}) / (12 * dx^2)
    """
    return (-np.roll(T, 2) + 16*np.roll(T, 1) - 30*T + 16*np.roll(T, -1) - np.roll(T, -2)) / (12 * dx**2)

In [11]:
def bilaplacian(T: np.ndarray, dx: float) -> np.ndarray:
    """
    Compute the biharmonic (bilaplacian) operator for a 1D array using two successive second-order Laplacians.

    Parameters
    ----------
    T : np.ndarray
        Array of values at cell centers.
    dx : float
        Grid spacing.

    Returns
    -------
    np.ndarray
        The biharmonic (bilaplacian) of T using second-order finite differences.

    Notes
    -----
    - This implementation uses the second-order Laplacian twice (laplacian_2nd).
    - Assumes periodic boundary conditions as in laplacian_2nd.
    """
    return laplacian_2nd(laplacian_2nd(T, dx), dx)

In [12]:
def laplacian_none(T: np.ndarray, dx: float) -> np.ndarray:
    """
    Return zero values for the Laplacian operator (i.e., no diffusion).

    Parameters
    ----------
    T : np.ndarray
        Array of values at cell centers.
    dx : float
        Grid spacing (ignored).

    Returns
    -------
    np.ndarray
        Array of zeros with the same shape as T.

    Notes
    -----
    - This function can be used to disable diffusion in the numerical scheme.
    - The dx parameter is ignored.
    """
    return np.zeros_like(T)

### Time-stepping

In [13]:
def rk3_step(T, rhs, dt):
    """
    Advance the solution by one time step using the third-order Runge-Kutta (RK3) integration scheme.

    Parameters
    ----------
    T : np.ndarray
        Array of the solution at the current time step.
    rhs : callable
        Function that computes the right-hand side (time derivative) given T.
    dt : float
        Time step size.

    Returns
    -------
    np.ndarray
        The updated solution array after one RK3 time step.

    Notes
    -----
    - This is a three-stage strong stability-preserving Runge-Kutta (SSP RK3) method.
    - The signature for `rhs` should be: rhs(T) -> np.ndarray (same shape as T).
    """
    R1 = rhs(T)
    T1 = T + dt * R1
    R2 = rhs(T1)
    T2 = 0.75 * T + 0.25 * (T1 + dt * R2)
    R3 = rhs(T2)
    T_new = (1/3) * T + (2/3) * (T2 + dt * R3)
    return T_new

In [14]:
def euler_step(T, rhs, dt):
    """
    Advance the solution by one time step using the (explicit) Euler integration scheme.

    Parameters
    ----------
    T : np.ndarray
        Array of the solution at the current time step.
    rhs : callable
        Function that computes the right-hand side (time derivative) given T.
    dt : float
        Time step size.

    Returns
    -------
    np.ndarray
        The updated solution array after one Euler time step.

    Notes
    -----
    - This is the explicit forward (Euler) method.
    - The signature for `rhs` should be: rhs(T) -> np.ndarray (same shape as T).
    """
    return T + dt * rhs(T)

### Combination

In [15]:
flux_scheme_dict = {
    "none": "No advection",
    "up1": "Up1 (1st order upstream)",
    "up3": "Up3 (3rd order upstream)",
    "cen2": "Cen2 (2nd order centered)",
    "cen4": "Cen4 (4th order centered)",
    "fct_cen2": "FCT (Cen2)",
    "fct_cen4": "FCT (Cen4)"
}

laplacian_operator_dict = {
    "none": ("No diffusion", laplacian_none),
    "lap2": ("2nd-order Laplacian", laplacian_2nd),
    "lap4": ("4th-order Laplacian", laplacian_4th),
    "bi": ("Bilaplacian", bilaplacian)
}

time_scheme_dict = {
    "rk3": "RK3",
    "euler": "Explicit Euler"
}

In [16]:
def run_advection_diffusion_simulation(
    nx: int,
    flux_scheme: str,
    nu: float,
    laplacian_func,
    CFL_adv: float,
    CFL_diff: float,
    theta_fct: float,
    time_final: float,
    time_scheme: str
):
    """
    Run an advection-diffusion simulation with specified numerical schemes and parameters.

    Parameters
    ----------
    nx : int
        Number of spatial grid points.
    flux_scheme : str
        Key for the advection flux scheme (e.g., 'none', 'up1', 'cen2', 'cen4', 'fct_cen2', 'fct_cen4').
    nu : float
        Diffusion coefficient.
    laplacian_func : callable
        Function for the Laplacian operator (e.g., laplacian_none, laplacian_2nd, laplacian_4th, bilaplacian).
    CFL_adv : float
        CFL number for advection (stability parameter).
    CFL_diff : float
        CFL number for diffusion (stability parameter).
    theta_fct : float
        Blending factor for FCT schemes (only used for 'fct_cen2' or 'fct_cen4').
    time_final : float
        Final simulation time.
    time_scheme : str
        Time-stepping scheme key ('rk3' or 'euler').

    Returns
    -------
    tuple
        (DOM, SOL_num, SOL_exact, dx, dt, nt, time_final)
        DOM : np.ndarray
            Spatial grid points.
        SOL_num : np.ndarray
            Numerical solution at final time.
        SOL_exact : np.ndarray
            Exact solution at final time.
        dx : float
            Grid spacing.
        dt : float
            Time step size.
        nt : int
            Number of time steps.
        time_final : float
            Final simulation time.

    Notes
    -----
    - Uses operator splitting (Lie): advection step then diffusion step each time step.
    - Assumes global variables: L (domain length), u (advection velocity), initial_condition (callable).
    - The function `flux_fn` is selected according to `flux_scheme`.
    - Periodic boundary conditions are assumed for advection and diffusion.
    - The time step `dt` is chosen based on stability for both advection and diffusion.
    """
    dx = L / nx
    dt_adv = CFL_adv * dx / abs(u) if (flux_scheme != "none" and abs(u) > 0) else np.inf
    if laplacian_func is laplacian_none or nu == 0.0:
        dt_diff = np.inf
    else:
        dt_diff = CFL_diff * dx**2 / (2 * nu)
    dt = min(dt_adv, dt_diff)
    nt = int(np.ceil(time_final / dt))
    dt = time_final / nt if nt > 0 else time_final
    DOM = np.linspace(0, L, nx, endpoint=False)
    SOL_num = initial_condition(DOM)
    flux_fn = {
        "none": compute_flux_none,
        "up1": compute_flux_up1,
        "up3": compute_flux_up3,
        "cen2": compute_flux_cen2,
        "cen4": compute_flux_cen4,
        "fct_cen2": lambda T, u, dx: compute_flux_fct(T, u, dx, base='cen2', theta=theta_fct),
        "fct_cen4": lambda T, u, dx: compute_flux_fct(T, u, dx, base='cen4', theta=theta_fct),
    }[flux_scheme]

    def adv_rhs(T):
        if flux_scheme == "none":
            return np.zeros_like(T)
        F = flux_fn(T, u, dx)
        return - (F[1:] - F[:-1]) / dx

    def diff_rhs(T):
        if laplacian_func is laplacian_none:
            return np.zeros_like(T)
        return nu * laplacian_func(T, dx)

    # Operator splitting (Lie): advection then diffusion
    for _ in range(nt):
        if time_scheme == "rk3":
            SOL_num = rk3_step(SOL_num, adv_rhs, dt)
            SOL_num = rk3_step(SOL_num, diff_rhs, dt)
        elif time_scheme == "euler":
            SOL_num = euler_step(SOL_num, adv_rhs, dt)
            SOL_num = euler_step(SOL_num, diff_rhs, dt)
        else:
            raise ValueError(f"Unknown time scheme: {time_scheme}")

    # Always compute exact at user-selected final time
    if flux_scheme == "none":
        SOL_exact = initial_condition(DOM)
    else:
        SOL_exact = initial_condition((DOM - u * time_final) % L)
    return DOM, SOL_num, SOL_exact, dx, dt, nt, time_final

## App

In [17]:
"""
Dash application for interactive 1D advection-diffusion equation exploration.

- Lets user select among advection, diffusion, and time stepping schemes.
- Supports solution plots and convergence analysis.
- Designed for coder-style clarity: includes type hints, docstrings, and key inline comments.

The following must be defined/imported elsewhere in your project:
    flux_scheme_dict: Dict[str, str]
        Dictionary mapping advection scheme keys to descriptive names.
    laplacian_operator_dict: Dict[str, Tuple[str, Any]]
        Dictionary mapping diffusion scheme keys to (description, operator function).
    time_scheme_dict: Dict[str, str]
        Dictionary mapping time stepping scheme keys to descriptive names.
    run_advection_diffusion_simulation: Callable[..., Tuple[np.ndarray, np.ndarray, np.ndarray, float, float, int, float]]
        Function for running the advection-diffusion simulation.
    initial_condition: Callable[[np.ndarray], np.ndarray]
        Function specifying the initial condition.
    L: float
        Domain length.
    u: float
        Advection velocity.
"""
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Interactive 1D Advection-Diffusion Explorer"),
    html.Div([
        # Sidebar controls
        html.Div([
            html.Label("Advection Discretization Schemes:"),
            dcc.Dropdown(
                id='flux-schemes',
                options=[{"label": v, "value": k} for k, v in flux_scheme_dict.items()],
                value=["cen2"], multi=True
            ),
            html.Label("Diffusion Discretization Schemes:"),
            dcc.Dropdown(
                id='diff-schemes',
                options=[{"label": v[0], "value": k} for k, v in laplacian_operator_dict.items()],
                value=["lap2"], multi=True
            ),
            html.Label("Time stepping schemes:"),
            dcc.Dropdown(
                id='time-schemes',
                options=[{"label": v, "value": k} for k, v in time_scheme_dict.items()],
                value=["rk3"], multi=True
            ),
            html.Label("CFL Number (advection):"),
            dcc.Slider(
                id='cfl-adv-slider', min=0.0, max=2.0, step=0.01, value=1.0,
                marks={0.0: "0.0", 1.0: "1.0", 2.0: "2.0"}
            ),
            html.Label("CFL Number (diffusion):"),
            dcc.Slider(
                id='cfl-diff-slider', min=0.0, max=2.0, step=0.01, value=1.0,
                marks={0.0: "0.0", 1.0: "1.0", 2.0: "2.0"}
            ),
            html.Label("Diffusion coefficient (nu):"),
            dcc.Slider(
                id='nu-slider', min=0.0, max=0.1, step=0.001, value=0.01,
                marks={0.0: "0.0", 0.05: "0.05", 0.1: "0.1"}
            ),
            html.Label("Number of grid cells (nx):"),
            dcc.Slider(
                id='nx-slider', min=20, max=400, step=10, value=100,
                marks={20: "20", 100: "100", 200: "200", 400: "400"}
            ),
            html.Label("Theta for FCT schemes:"),
            dcc.Slider(
                id='theta-slider', min=0.0, max=1.0, step=0.05, value=0.5,
                marks={0.0: "0.0", 0.5: "0.5", 1.0: "1.0"}
            ),
            html.Label("Final time (T):"),
            dcc.Slider(
                id='final-time-slider', min=0.1, max=5.0, step=0.01, value=1.0,
                marks={0.1: "0.1", 1.0: "1.0", 2.0: "2.0", 5.0: "5.0"}
            ),
            html.Br(),
            html.Button("Recompute convergence", id='recompute-conv-btn', n_clicks=0),
            dcc.Store(id='convergence-cache')
        ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': 20}),

        # Main content: tabs for solution and convergence
        html.Div([
            dcc.Tabs(id="tabs", value='tab-sol', children=[
                dcc.Tab(label='Solution plot', value='tab-sol'),
                dcc.Tab(label='Convergence analysis', value='tab-conv'),
            ]),
            html.Div(
                dcc.Loading(
                    id="loading-conv",
                    type="dot",
                    children=html.Div(id='tabs-content')
                ),
                style={"height": "600px"}
            )
        ], style={'width': '65%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ])
])

@app.callback(
    Output('tabs-content', 'children'),
    Output('convergence-cache', 'data'),
    Input('flux-schemes', 'value'),
    Input('diff-schemes', 'value'),
    Input('time-schemes', 'value'),
    Input('cfl-adv-slider', 'value'),
    Input('cfl-diff-slider', 'value'),
    Input('nu-slider', 'value'),
    Input('nx-slider', 'value'),
    Input('theta-slider', 'value'),
    Input('final-time-slider', 'value'),
    Input('tabs', 'value'),
    Input('recompute-conv-btn', 'n_clicks'),
    State('convergence-cache', 'data')
)
def update_plots(
    flux_schemes: List[str],
    diff_schemes: List[str],
    time_schemes: List[str],
    CFL_adv: float,
    CFL_diff: float,
    nu: float,
    nx: int,
    theta_fct: float,
    time_final: float,
    tab: str,
    recalc_clicks: int,
    cache_data: Any
) -> Tuple[Any, Any]:
    """
    Dash callback: updates plots and convergence cache based on user selections.

    Parameters
    ----------
    flux_schemes : List[str]
        Selected advection schemes.
    diff_schemes : List[str]
        Selected diffusion schemes.
    time_schemes : List[str]
        Selected time stepping schemes.
    CFL_adv : float
        CFL number for advection.
    CFL_diff : float
        CFL number for diffusion.
    nu : float
        Diffusion coefficient.
    nx : int
        Number of grid points.
    theta_fct : float
        Blending for FCT schemes.
    time_final : float
        Final time.
    tab : str
        Selected tab ("tab-sol" or "tab-conv").
    recalc_clicks : int
        Number of times "recompute convergence" has been pressed.
    cache_data : Any
        Cached convergence traces.

    Returns
    -------
    Tuple
        (Tab content, possibly updated cache_data)
    """

    # Require at least one scheme of each type.
    if not flux_schemes or not diff_schemes or not time_schemes:
        return html.Div("Please select at least one advection, one diffusion, and one time stepping scheme."), cache_data

    # Solution plot tab
    if tab == 'tab-sol':
        fig = go.Figure()
        for flux_key in flux_schemes:
            for lap_key in diff_schemes:
                lap_name, lap_func = laplacian_operator_dict[lap_key]
                for time_key in time_schemes:
                    # Run simulation for current parameter set
                    DOM, SOL_num, SOL_exact, dx, dt, nt, _ = run_advection_diffusion_simulation(
                        nx=nx,
                        flux_scheme=flux_key,
                        nu=nu,
                        laplacian_func=lap_func,
                        CFL_adv=CFL_adv,
                        CFL_diff=CFL_diff,
                        theta_fct=theta_fct,
                        time_final=time_final,
                        time_scheme=time_key
                    )
                    fig.add_trace(go.Scatter(
                        x=DOM,
                        y=SOL_num,
                        mode='lines',
                        name=f"{flux_scheme_dict[flux_key]} + {lap_name} + {time_scheme_dict[time_key]}"
                    ))
        # Reference: analytical solution
        DOM = np.linspace(0, L, nx, endpoint=False)
        if "none" in flux_schemes:
            SOL_exact = initial_condition(DOM)
            fig.add_trace(go.Scatter(
                x=DOM,
                y=SOL_exact,
                mode='lines',
                name='Exact (no advection)',
                line=dict(dash='dash', color='black')
            ))
        else:
            SOL_exact = initial_condition((DOM - u * time_final) % L)
            fig.add_trace(go.Scatter(
                x=DOM,
                y=SOL_exact,
                mode='lines',
                name='Exact (advection only)',
                line=dict(dash='dash', color='black')
            ))
        fig.update_layout(
            title=f"1D Advection-Diffusion: Selected Schemes<br>ν={nu}, CFL_adv={CFL_adv}, CFL_diff={CFL_diff}, nx={nx}, T={time_final}",
            xaxis_title='x',
            yaxis_title='Tracer T(x)',
            legend_title='Scheme combinations',
            template='plotly_white'
        )
        return dcc.Graph(figure=fig), cache_data

    # Convergence analysis tab
    if tab == 'tab-conv':
        ctx = callback_context
        triggered = [t['prop_id'] for t in ctx.triggered]
        recalc = 'recompute-conv-btn.n_clicks' in triggered or cache_data is None

        if recalc and recalc_clicks > 0:
            # Recompute convergence for several grid resolutions
            nxs = [5, 10, 20, 40, 80, 160, 320]
            all_traces = []
            for flux_key in flux_schemes:
                for lap_key in diff_schemes:
                    lap_name, lap_func = laplacian_operator_dict[lap_key]
                    for time_key in time_schemes:
                        DX, ERROR = [], []
                        for nx_c in nxs:
                            DOM, SOL_num, _, dx, dt, nt, _ = run_advection_diffusion_simulation(
                                nx=nx_c,
                                flux_scheme=flux_key,
                                nu=nu,
                                laplacian_func=lap_func,
                                CFL_adv=CFL_adv,
                                CFL_diff=CFL_diff,
                                theta_fct=theta_fct,
                                time_final=time_final,
                                time_scheme=time_key
                            )
                            # Compute max error vs exact
                            if flux_key == "none":
                                error = np.linalg.norm(SOL_num - initial_condition(DOM), np.inf)
                            else:
                                SOL_exact_actual = initial_condition((DOM - u * time_final) % L)
                                error = np.linalg.norm(SOL_num - SOL_exact_actual, np.inf)
                            DX.append(dx)
                            ERROR.append(error)
                        all_traces.append({
                            "x": DX, "y": ERROR,
                            "name": f"{flux_scheme_dict[flux_key]} + {lap_name} + {time_scheme_dict[time_key]}"
                        })
            ref_dx = np.array(DX)
            # Reference slopes for order-of-accuracy
            all_traces += [
                {"x": ref_dx, "y": ref_dx, "name": "1st order", "line": dict(dash='dash', color='gray')},
                {"x": ref_dx, "y": ref_dx**2, "name": "2nd order", "line": dict(dash='dot', color='gray')},
                {"x": ref_dx, "y": ref_dx**3, "name": "3rd order", "line": dict(dash='dashdot', color='gray')}
            ]
            cache_data = all_traces  # update cache
            fig2 = go.Figure()
            for tr in all_traces:
                fig2.add_trace(go.Scatter(x=tr["x"], y=tr["y"],
                                          mode='lines+markers' if "line" not in tr else 'lines',
                                          name=tr["name"], line=tr.get("line", {})))
            fig2.update_layout(
                title=f'Convergence Analysis (L∞-error vs Δx), T={time_final}',
                xaxis_type='log',
                yaxis_type='log',
                xaxis_title='Δx',
                yaxis_title='L∞ Error',
                template='plotly_white'
            )
            return dcc.Graph(figure=fig2), cache_data
        elif cache_data:
            # Use cached convergence plot
            fig2 = go.Figure()
            for tr in cache_data:
                fig2.add_trace(go.Scatter(x=tr["x"], y=tr["y"],
                                          mode='lines+markers' if "line" not in tr else 'lines',
                                          name=tr["name"], line=tr.get("line", {})))
            fig2.update_layout(
                title=f'Convergence Analysis (L∞-error vs Δx), T={time_final}',
                xaxis_type='log',
                yaxis_type='log',
                xaxis_title='Δx',
                yaxis_title='L∞ Error',
                template='plotly_white'
            )
            return dcc.Graph(figure=fig2), cache_data
        else:
            # No cache and not triggered
            return html.Div("Click 'Recompute convergence' to compute convergence analysis."), cache_data

    # Default: show recommendation to trigger convergence
    return html.Div("Click 'Recompute convergence' to compute convergence analysis."), cache_data

if __name__ == "__main__":
    app.run(debug=False, mode="inline")